In [ ]:
#Juan David Useche y Juan Sebastián Nuncira

In [14]:
import tkinter as tk
from tkinter import ttk
import sqlite3
import pandas as pd
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from tkinter.messagebox import showerror, showwarning, showinfo


class App(tk.Tk): 
    def __init__(self):
        super().__init__()
        
        style = ttk.Style()
        style.configure('My.TLabel', background='#F2E3D5',foreground='#012E40')
        style.configure('My.TButton',background='#3CA6A6',foreground='#012E40')
        self.iconbitmap("FINWER.ico")
        self.title("Ventana Principal")

        # Crear el controlador de pestañas
        self.controlador_pestanas = ttk.Notebook(self)
        self.controlador_pestanas.pack(fill="both", expand=True)

        # Crear las pestañas
        self.pestaña1 = tk.Frame(self.controlador_pestanas)
        self.pestaña2 = tk.Frame(self.controlador_pestanas)
        self.controlador_pestanas.add(self.pestaña1, text="Iniciar sesión")
        self.controlador_pestanas.add(self.pestaña2, text="Registrarse")
        
        # Crear tabla de usuarios
        conn = sqlite3.connect('base_registro.db')
        cursor = conn.cursor()
        query = """
        CREATE TABLE IF NOT EXISTS usuarios (
            usuario TEXT NOT NULL,
            contraseña TEXT NOT NULL
        );
        """
        cursor.execute(query)
        conn.commit()
        
         #Crear table para ingresos
        conn = sqlite3.connect('informacion_financiera.db')
        cursor = conn.cursor()

        query = """
        CREATE TABLE IF NOT EXISTS ingresos (
            tipo_ingreso CHAR(15),
            monto_ingreso DECIMAL (1000000000000000,4)
        );
        """
        cursor.execute(query)
        conn.commit()
        conn.close()
        
        #Crear table para gastos
        conn = sqlite3.connect('informacion_financiera.db')
        cursor = conn.cursor()

        query = """
        CREATE TABLE IF NOT EXISTS gastos (
            tipo_gasto CHAR(15),
            monto_gasto DECIMAL (1000000000000000,4)
        );
        """
        cursor.execute(query)
        conn.commit()
        conn.close()
        
        #crear table balance
        conn = sqlite3.connect('informacion_financiera.db')
        cursor = conn.cursor()
        query = """ CREATE TABLE IF NOT EXISTS balance (
            fecha_balance DATE,
            ingreso_neto DECIMAL (1000000000000000,4) ); """
        cursor.execute(query)
        conn.commit()
        conn.close()
    
        # Botón para ir a la Pestaña 1
        self.textonombre1 = ttk.Label(self.pestaña1, text="Nombre de usuario", font=('Helvetica', 10), style='My.TLabel')
        self.textonombre1.pack(pady=5)
        self.entryinicio1 = ttk.Entry(self.pestaña1, style='My.TButton')
        self.entryinicio1.pack(pady=5)
        self.textocontraseña1 = ttk.Label(self.pestaña1, text="Contraseña", font=('Helvetica', 10), style='My.TLabel')
        self.textocontraseña1.pack(pady=5)
        self.entrycontraseña1 = ttk.Entry(self.pestaña1, style='My.TButton',show='*')
        self.entrycontraseña1.pack(pady=5)
        self.boton_pestaña1 = tk.Button(self.pestaña1, text="Iniciar sesión", command=self.verificar_credenciales)
        self.boton_pestaña1.pack(pady=10)

        # Botón para ir a la Pestaña 2
        self.textonombre2 = ttk.Label(self.pestaña2, text="Nombre de usuario", font=('Helvetica', 10), style='My.TLabel')
        self.textonombre2.grid(pady=5, row=0)
        self.entryinicio = ttk.Entry(self.pestaña2, style='My.TButton')
        self.entryinicio.grid(pady=5, row=1)
        self.textocontraseña2 = ttk.Label(self.pestaña2, text="Contraseña", font=('Helvetica', 10), style='My.TLabel')
        self.textocontraseña2.grid(pady=5, row=2)
        self.entrycontraseña = ttk.Entry(self.pestaña2, style='My.TButton')
        self.entrycontraseña.grid(pady=5, row=3)
        self.boton_pestaña2 = tk.Button(self.pestaña2, text="Registrarse", command=self.creacion_usuario)
        self.boton_pestaña2.grid(pady=20, row=4)
        
    def verificar_credenciales(self):
        usuario = self.entryinicio1.get()
        contraseña = self.entrycontraseña1.get()
       
        if self.verificar_credenciales_bd(usuario, contraseña):
            self.main_frame = ttk.Frame(self)
            self.main_frame.pack(fill=tk.BOTH, expand=True)
            self.mostrar_home()
        else:
            showerror('Error','Credenciales incorrectos')
            
    def verificar_credenciales_bd(self, usuario, contraseña):
        conn = sqlite3.connect('base_registro.db')
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM usuarios WHERE usuario=? AND contraseña=?", (usuario, contraseña))
        count = cursor.fetchone()[0]
        conn.close()
        return count > 0
            
    def creacion_usuario(self):
        usuario = self.entryinicio.get()
        contraseña = self.entrycontraseña.get()
        conn = sqlite3.connect('base_registro.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO usuarios (usuario, contraseña) VALUES (?, ?)", (usuario, contraseña))
        conn.commit()
        conn.close()
        self.controlador_pestanas.select(self.pestaña1)
    
    def clear_frame(self, frame):
        for widget in frame.winfo_children():
            widget.destroy()
        
        self.cita = ""
        #cita de mentalidad
    def obtener_cita_financiera(self):
        api_key = 'KOILM8JOCTX6JWE9'
        url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=MSFT&apikey={api_key}'
        response = requests.get(url)
        data = response.json()
        quote = data['Global Quote']['05. price']
        cita = quote  # Asignar el valor de la cita a la variable cita
        return cita

    def mostrar_home(self):
        self.clear_frame(self.controlador_pestanas)
        self.clear_frame(self.main_frame)
        style = ttk.Style()
        style.configure('My.TFrame', background='#F2E3D5')
        #frame ingresos
        self.frameingresos=ttk.Frame(self.main_frame,height=200,width=600,borderwidth=5,relief="raised",style='My.TFrame')
        self.frameingresos.grid(column=0,row=0,pady=10,padx=10,sticky="nsew")
        #frame gastos
        self.framegastos=ttk.Frame(self.main_frame,height=200,width=600,borderwidth=5,relief="raised",style='My.TFrame')
        self.framegastos.grid(column=1,row=0,pady=10,padx=10,sticky="nsew")
        #frame comercio
        self.framecomercio=ttk.Frame(self.main_frame,height=200,width=600,borderwidth=5,relief="raised",style='My.TFrame')
        self.framecomercio.grid(column=0,row=1,pady=10,padx=10,sticky="nsew")
        #framebalance
        self.framebalance=ttk.Frame(self.main_frame,height=200,width=600,borderwidth=5,relief="raised",style='My.TFrame')
        self.framebalance.grid(column=1,row=1,pady=10,padx=10,sticky="nsew")
        #framementalidad
        self.framementalidad=ttk.Frame(self.main_frame,height=200,width=600,borderwidth=5,relief="raised",style='My.TFrame')
        self.framementalidad.grid(column=0,row=2,pady=10,padx=10,sticky="nsew")
        #frameconsejos
        self.frameconsejos=ttk.Frame(self.main_frame,height=200,width=600,borderwidth=5,relief="raised",style='My.TFrame')
        self.frameconsejos.grid(column=1,row=2,pady=10,padx=10,sticky="nsew")

        #Contenido ingresos
        style = ttk.Style()
        style.configure('My.TLabel', background='#F2E3D5',foreground='#012E40')
        style=ttk.Style()
        style.configure('My.TButton',background='#3CA6A6',foreground='#012E40')
        self.text1frameingresos=ttk.Label(self.frameingresos,text="Ingresos",style='My.TLabel',font=("calibri",15)).grid(padx=90)
        
        self.text1frameingresos=ttk.Label(self.frameingresos,text="Presiona para digitar tus ingresos",style='My.TLabel',font=("calibri",10,)).grid()
        self.text1frameingresos=ttk.Label(self.frameingresos,text="",style='My.TLabel').grid()

        self.route1_button = ttk.Button(self.frameingresos, text="Ingresos",style='My.TButton', command=self.ruta1)
        self.route1_button.grid()

        #Contenido gastos
        self.text1framegastos=ttk.Label(self.framegastos,text="Gastos",style='My.TLabel',font=("calibri",15)).grid()
        self.text1framegastos=ttk.Label(self.framegastos,text="Presiona para digitar tus gastos",style='My.TLabel',font=("calibri",10)).grid()
        self.text1framegastos=ttk.Label(self.framegastos,text="",style='My.TLabel').grid

        self.route2_button = ttk.Button(self.framegastos, text="Gastos",style='My.TButton', command=self.ruta2)
        self.route2_button.grid(pady=10)

        #Contenido comercio
        self.text1framecomercio=ttk.Label(self.framecomercio,text="Comercio",style='My.TLabel',font=("calibri",15)).grid()
        self.text1framecomercio=ttk.Label(self.framecomercio,text="Presiona para visualizar datos de tu comercio",style='My.TLabel',font=("calibri",10)).grid()
        self.text1framecomercio=ttk.Label(self.framecomercio,text="",style='My.TLabel').grid
        self.route1_button = ttk.Button(self.framecomercio, text="Mi tienda",style='My.TButton', command=self.ruta3)
        self.route1_button.grid(pady=10)

        #Contenido Balance
        self.text1framebalance=ttk.Label(self.framebalance,text="Balance",style='My.TLabel',font=("calibri",15)).grid()
        self.text1framebalance=ttk.Label(self.framebalance,text="Presiona para visualizar tu balance",style='My.TLabel',font=("calibri",10)).grid()
        self.text1framebalance=ttk.Label(self.framebalance,text="",style='My.TLabel').grid

        self.route2_button = ttk.Button(self.framebalance, text="Mi balance",style='My.TButton', command=self.comandos_balance)
        self.route2_button.grid(pady=10)

        # Contenido mentalidad
        self.text1framementalidad = ttk.Label(self.framementalidad, text="Mentalidad", style='My.TLabel', font=("calibri", 15)).grid(padx=90)
        self.cita_financiera = ttk.Label(self.framementalidad, text=f"{self.cita}", font=('Helvetica', 10), style='My.TLabel')
        self.cita_financiera.grid(pady=5)
        self.text1framementalidad = ttk.Label(self.framementalidad, text="", style='My.TLabel').grid()

        #Contenido consejos
        self.text1frameconsejos=ttk.Label(self.frameconsejos,text="Consejos",style='My.TLabel',font=("calibri",15)).grid()
        self.text1frameconsejos=ttk.Label(self.frameconsejos,text="Presiona para ver consejos de los mejores",style='My.TLabel',font=("calibri",10)).grid()
        self.text1frameconsejos=ttk.Label(self.frameconsejos,text="",style='My.TLabel').grid
        self.route1_button = ttk.Button(self.frameconsejos, text="Mis consejos",style='My.TButton', command=self.ruta6)
        self.route1_button.grid(pady=10)
        
        self.balance_ig (extraer de la formula)
        # Flujo de efectivo de la inversión a lo largo de los 10 años (supongamos un flujo de efectivo constante)
        cash_flows = [balance_ig] * 10

        # Agregar el valor inicial del balance_ig al flujo de efectivo (si es necesario)
        cash_flows[0] += balance_ig

        # Calcular el VPN (Valor Presente Neto)
        discount_rate = 0.07  # Rentabilidad del 7%
        npv = np.npv(discount_rate, cash_flows)

        # Calcular la TIR (Tasa Interna de Retorno)
        irr = np.irr(cash_flows)
        
    def ruta1(self):
        style=ttk.Style()
        style.configure('My.TCombobox',background='#3CA6A6',foreground='#012E40')

        #Borro todos los widgets
        self.clear_frame(self.main_frame)
        self.main_frame.configure(style='My.TFrame')


        #Coloco los de la página en cuestión
        self.label = ttk.Label(self.main_frame, text="Ingresos",style='My.TLabel', font=("Helvetica", 30))
        self.label.pack(pady=20)

        self.widget1 = ttk.Label(self.main_frame, text="Digita el monto de tus ingresos",font=('Helvetica',15),style='My.TLabel')
        self.widget1.pack(pady=10)
        self.entryingresos = ttk.Entry(self.main_frame,style='My.TButton')
        self.entryingresos.pack(pady=10)

        self.widget2 = ttk.Label(self.main_frame, text="Digita la categoría de tus ingresos",font=('Helvetica',15),style='My.TLabel')
        self.widget2.pack(pady=10)
        selected_option = tk.StringVar()

        self.options_list_income = ttk.Combobox(self.main_frame, textvariable=selected_option,style='My.TCombobox')
        self.options_list_income["values"] = ["Sueldo", "Arriendos", "Bonificaciones","Venta","Dividendos"]
        self.options_list_income.pack()

        print(selected_option.get())
        self.home_button = ttk.Button(self.main_frame, text="Volver",style='My.TButton', command=self.mostrar_home)
        self.home_button.pack(pady=8)
        self.save_button1 = ttk.Button(self.main_frame, text="Guardar", style='My.TButton', command= self.obtener_datos_ingresos)
        self.save_button1.pack(pady=5)
        #Back-end ingresos
        
    def obtener_datos_ingresos(self):
        datos_ingresos = [self.entryingresos.get(), self.options_list_income.get()]
        conn = sqlite3.connect('informacion_financiera.db')
        cursor = conn.cursor()        
        cursor.execute('INSERT INTO ingresos (monto_ingreso, tipo_ingreso) VALUES (?, ?)', datos_ingresos)
        conn.commit()
        conn.close()
        self.entryingresos.delete(0, 'end')
        self.options_list_income.set('')

    def ruta2(self):
        #Borro todos los widgets
        self.clear_frame(self.main_frame)

         #Coloco los de la página en cuestión
        self.label = ttk.Label(self.main_frame, text="Gastos",style='My.TLabel', font=("Helvetica", 30))
        self.label.pack(pady=20)

        self.widget1 = ttk.Label(self.main_frame, text="Digita tus gastos",font=('Helvetica',15),style='My.TLabel')
        self.widget1.pack(pady=10)
        self.entrygastos = ttk.Entry(self.main_frame,style='My.TButton')
        self.entrygastos.pack(pady=10)

        self.widget2 = ttk.Label(self.main_frame, text="Digita la categoría de tus gastos",font=('Helvetica',15),style='My.TLabel')
        self.widget2.pack(pady=10)
        selected_option = tk.StringVar()

        self.options_list_gastos = ttk.Combobox(self.main_frame, textvariable=selected_option,style='My.TCombobox')
        self.options_list_gastos["values"] = ["gastos fijos", "gastos variables", "gastos hormiga", "gastos discrecionales"]
        self.options_list_gastos.pack()

        print(selected_option.get())
        self.home_button = ttk.Button(self.main_frame, text="Volver",style='My.TButton', command=self.mostrar_home)
        self.home_button.pack(pady=8)
        self.save_button = ttk.Button(self.main_frame, text="Guardar",style='My.TButton', command = self.obtener_datos_gastos)
        self.save_button.pack(pady=5)
        
    def obtener_datos_gastos(self):
        datos_gastos = [self.entrygastos.get(), self.options_list_gastos.get()]
        conn = sqlite3.connect('informacion_financiera.db')
        cursor = conn.cursor()        
        cursor.execute('INSERT INTO gastos (monto_gasto, tipo_gasto) VALUES (?, ?)', datos_gastos)
        conn.commit()
        conn.close()
        self.entrygastos.delete(0, 'end')
        self.options_list_gastos.set('')

    def ruta3(self):
        #Borro todos los widgets
        self.clear_frame(self.main_frame)
        #Coloco los de la página en cuestión
        #Frames
        #FrameC
        self.frameCcomercio=ttk.Frame(self.main_frame,borderwidth=5,relief="raised",style='My.TFrame')
        self.frameCcomercio.grid(pady=0, padx=0, row=0, columnspan=2, sticky='nsew')
        
        #Frame A
        self.frameAcomercio=ttk.Frame(self.main_frame,borderwidth=5,relief="raised",style='My.TFrame')
        self.frameAcomercio.grid(row=1,pady=0,padx=0,sticky="nsew")
        
        #Frame AA
        self.frameAAcomercio=ttk.Frame(self.frameAcomercio,borderwidth=5,relief='raised',style='My.TFrame')
        self.frameAAcomercio.grid(column=0,sticky='nsew')
        
        #FrameAB
        self.frameABcomercio=ttk.Frame(self.frameAcomercio,borderwidth=5,relief='raised',style='My.TFrame')
        self.frameABcomercio.grid(column=1,row=0,sticky='nsew')
        
       
        #Frame B
        self.frameBcomercio=ttk.Frame(self.main_frame,borderwidth=5,relief="raised",style='My.TFrame')
        self.frameBcomercio.grid(row=2,pady=0,padx=0,sticky="nsew")
       
        #Contenido Frame C
        self.home_buttonAA = ttk.Button(self.frameCcomercio, text="Volver",style='My.TButton', command=self.mostrar_home)
        self.home_buttonAA.grid(pady=5,sticky='w')#poner flecha de volver
        
        self.label = ttk.Label(self.frameCcomercio, text="Mi comercio",style='My.TLabel', font=("Helvetica", 25))
        self.label.grid(pady=5, padx=5, row=1, column=1,columnspan=2, sticky="nsew")
        
        #Contenido Frame A    
        #AA
        
        self.widget1 = ttk.Label(self.frameAAcomercio, text="Nombre de producto",font=('Helvetica',10),style='My.TLabel')
        self.widget1.grid(pady=5)
        self.entrynomproduct = ttk.Entry(self.frameAAcomercio,style='My.TButton')
        self.entrynomproduct.grid(pady=5)
        self.widget2 = ttk.Label(self.frameAAcomercio, text="Precio",font=('Helvetica',10),style='My.TLabel')
        self.widget2.grid(pady=10)
        self.entryprecios = ttk.Entry(self.frameAAcomercio,style='My.TButton')
        self.entryprecios.grid(pady=5)#Falta poner que solo admita números para hacer operaciones matemáticas

        #ABA
        #self.label = ttk.Label(self.frameABcomercio, text="",style='My.TLabel', font=("Helvetica", 45))
        #self.label.grid(pady=20,row=0)
        self.nombreproducto = ttk.Label(self.frameABcomercio, text="Unidades de producto",font=('Helvetica',10),style='My.TLabel')
        self.nombreproducto.grid( pady=5, padx=5,row=1,columnspan=2)
        self.entrynomproduct = ttk.Entry(self.frameABcomercio,style='My.TButton')
        self.entrynomproduct.grid(pady=5, padx=5,row=2,columnspan=2)
        
        #ABB
        self.nombreproducto = ttk.Label(self.frameABcomercio, text="Descuento",font=('Helvetica',10),style='My.TLabel')
        self.nombreproducto.grid(pady=5,padx=5,row=3,column=0)
        self.entrynomproduct = ttk.Entry(self.frameABcomercio,style='My.TButton')
        self.entrynomproduct.grid(pady=5,padx=5,row=4,column=0)
        
        #ABC
        self.nombreproducto = ttk.Label(self.frameABcomercio, text="Unidades con descuento",font=('Helvetica',10),style='My.TLabel')
        self.nombreproducto.grid(pady=5,padx=5,row=3,column=1)
        self.entrynomproduct = ttk.Entry(self.frameABcomercio,style='My.TButton')
        self.entrynomproduct.grid(pady=5,padx=5,row=4,column=1)

        self.save_buttonABC = ttk.Button(self.frameABcomercio, text="Guardar",style='My.TButton')
        self.save_buttonABC.grid(pady=5,row=5,column=1,sticky='e')
        
    def ruta4(self):
        #Borro todos los widgets
        self.clear_frame(self.main_frame)
        #Coloco los de la página en cuestión
        #Frames
        #FrameC
        self.frameCbalance=ttk.Frame(self.main_frame,borderwidth=5,relief="raised",style='My.TFrame')
        self.frameCbalance.grid(pady=0, padx=0, row=0, columnspan=20,sticky='ew')
        
        #Frame A
        self.frameAbalance=ttk.Frame(self.main_frame,borderwidth=5,relief="raised",style='My.TFrame')
        self.frameAbalance.grid(row=1,pady=0,padx=0,columnspan=2,sticky='ew')
        
        #Frame AA (ingresos)
        self.frameAAbalance=ttk.Frame(self.frameAbalance,borderwidth=5,relief='raised',style='My.TFrame')
        self.frameAAbalance.grid(column=0,pady=1,padx=1)
        
        #FrameAB (egresos)
        self.frameABbalance=ttk.Frame(self.frameAbalance,borderwidth=5,relief='raised',style='My.TFrame')
        self.frameABbalance.grid(column=1,row=0,pady=1,padx=1,columnspan=2)
        
        #Frame B
        self.frameBbalance=ttk.Frame(self.main_frame,borderwidth=5,relief="raised",style='My.TFrame')
        self.frameBbalance.grid(row=2,pady=0,padx=0,sticky="ew",columnspan=2)
        
        #Frame AC (ingresos netos)
        self.frameACbalance=ttk.Frame(self.frameAbalance,borderwidth=5,relief='raised',style='My.TFrame')
        self.frameACbalance.grid(column=2,row=0,pady=1,padx=1,columnspan=3)
       
        #Contenido Frame C
        self.home_buttonAA = ttk.Button(self.frameCbalance, text="Volver",style='My.TButton', command=self.mostrar_home)
        self.home_buttonAA.grid(pady=5,sticky='w')#poner flecha de volver
        
        self.label = ttk.Label(self.frameCbalance, text="Mi balance",style='My.TLabel', font=("Helvetica", 25))
        self.label.grid(pady=5, padx=5, row=1, column=0,columnspan=2, sticky="nsew")
        
        #Contenido ingresos balance  
        
        self.widgetbalance = ttk.Label(self.frameAAbalance, text="Ingresos",font=('Helvetica',20),style='My.TLabel')
        self.widgetbalance.grid(pady=5,sticky='w')#Print los ingresos registrados antes
        # Crear gráfica de ingresos
        conn = sqlite3.connect('informacion_financiera.db')
        df_ingresos = pd.read_sql_query("SELECT * FROM ingresos", conn)
        conn.close()
        
        if not df_ingresos.empty:
            plt.figure(figsize=(5, 5))
            plt.pie(df_ingresos['monto_ingreso'], labels=df_ingresos['tipo_ingreso'], autopct='%1.1f%%')
            plt.title('Ingresos')
            plt.axis('equal')
            plt.savefig('grafica_ingresos.png')  # Guardar la gráfica en un archivo
            
            # Mostrar la gráfica en el frame
            self.grafica_ingresos = tk.PhotoImage(file='grafica_ingresos.png')
            self.label_grafica_ingresos = tk.Label(self.frameAAbalance, image=self.grafica_ingresos)
            self.label_grafica_ingresos.grid(row=3, column=0, pady=10)
            
        #Contenido gastos balance
        
        self.nombrebalance = ttk.Label(self.frameABbalance, text="Gastos",font=('Helvetica',20),style='My.TLabel')
        self.nombrebalance.grid( pady=5, padx=15,row=1,columnspan=2)#print los gastos registrados
        
        # Crear gráfica de gastos
        conn = sqlite3.connect('informacion_financiera.db')
        df_gastos = pd.read_sql_query("SELECT * FROM gastos", conn)
        conn.close()
        
        if not df_gastos.empty:
            plt.figure(figsize=(5, 5))
            plt.pie(df_gastos['monto_gasto'], labels=df_gastos['tipo_gasto'], autopct='%1.1f%%')
            plt.title('Gastos')
            plt.axis('equal')
            plt.savefig('grafica_gastos.png')  # Guardar la gráfica en un archivo
            
            # Mostrar la gráfica en el frame
            self.grafica_gastos = tk.PhotoImage(file='grafica_gastos.png')
            self.label_grafica_gastos = tk.Label(self.frameABbalance, image=self.grafica_gastos)
            self.label_grafica_gastos.grid(row=3, column=0, pady=10)    
            
        #grafica balance final
    def grafica_tendencial(self):
        conn = sqlite3.connect('informacion_financiera.db')
        cursor = conn.cursor()

        cursor.execute("SELECT fecha_balance, ingreso_neto FROM balance ORDER BY fecha_balance")
        registros = cursor.fetchall()

        fechas = []
        ingresos_netos = []

        for registro in registros:
            fechas.append(registro[0])
            ingresos_netos.append(registro[1])

        conn.close()

        plt.plot(fechas, ingresos_netos)
        plt.xlabel('Fecha')
        plt.ylabel('Ingreso Neto')
        plt.title('Gráfica de ahorros')
        plt.xticks(rotation=45)
        plt.show(self.frameACbalance)
            
        #balance final
    def balance_final(self):
        conn = sqlite3.connect('informacion_financiera.db')
        cursor = conn.cursor()

        cursor.execute("SELECT SUM(monto_ingreso) FROM ingresos")
        total_ingreso = cursor.fetchone()[0]

        cursor.execute("SELECT SUM(monto_gasto) FROM gastos")
        total_gasto = cursor.fetchone()[0]
        
        balance_ig = total_ingreso - total_gasto
        cursor.execute("SELECT * FROM balance ORDER BY ingreso_neto DESC LIMIT 1") 
        ultimo_dato = cursor.fetchone()
        fecha_actual = datetime.now().date()
        if balance_ig != ultimo_dato:
            cursor.execute('INSERT INTO balance (fecha_balance, ingreso_neto) VALUES (?, ?)', (fecha_actual, balance_ig))
        else:
            pass
        conn.commit()
        conn.close()
        
    def comandos_balance(self):
        self.ruta4()
        self.balance_final()
    
    def ruta6(self):
        #Borro todos los widgets
        self.clear_frame(self.main_frame)
        #Coloco los de la página en cuestión
        self.label = ttk.Label(self.main_frame, text="Ruta 6", font=("Helvetica", 30))
        self.label.pack(pady=20)

        self.widget1 = ttk.Label(self.main_frame, text="Widget 1 en Ruta 6")
        self.widget1.pack(pady=10)

        self.widget2 = ttk.Label(self.main_frame, text="Widget 2 en Ruta 6")
        self.widget2.pack(pady=10)

        self.home_button = ttk.Button(self.main_frame, text="Volver a la Página Principal", command=self.mostrar_home)
        self.home_button.pack(pady=20)
app = App()
app.mainloop()

SyntaxError: invalid syntax (1591283065.py, line 224)

In [26]:
import sqlite3

conn = sqlite3.connect('informacion_financiera.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM gastos")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

('gastos variables', 13212)
('gastos variables', 654)
('gastos hormiga', 4214)


In [27]:
import sqlite3

conn = sqlite3.connect('informacion_financiera.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM ingresos")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

('Sueldo', 1234567890)
('Arriendos', 42114)
('Dividendos', 923456789)


In [28]:
import sqlite3

conn = sqlite3.connect('informacion_financiera.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM balance")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
('2023-05-29', 2158048713)
